In [39]:
import cv2
import numpy as np
import pytesseract
import re

In [40]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Adjust path as needed


In [41]:
img= cv2.imread('C:\\Users\\Aman\\Desktop\\thesisproject\\car_image\\Ethiopia_EV1.jpg')

🔍 What is a bilateral filter?
A bilateral filter smooths the image like a Gaussian blur but preserves edges, which is crucial when detecting license plates (you want to blur noise but keep edges like letters and borders).

d = 11: Diameter of each pixel neighborhood. Larger = more blur.

sigmaColor = 17: How much colors can differ before they’re considered different.

Larger value means more colors get blurred together.

sigmaSpace = 17: How far pixels can be from the center pixel to influence the result.

Larger means a bigger neighborhood is considered for blurring.

⚙️ These values were chosen experimentally — they offer a good balance for removing noise while preserving license plate edges. You can tweak them based on your i

In [42]:
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Apply bilateral filter (preserves edges better than GaussianBlur)
# gray = cv2.bilateralFilter(gray, 11, 17, 17)

# # Increase contrast using histogram equalization
# gray = cv2.equalizeHist(gray)

# # Sharpen image (to counteract blurriness)
# kernel = np.array([[0, -1, 0],
#                    [-1, 5,-1],
#                    [0, -1, 0]])
# gray = cv2.filter2D(gray, -1, kernel)


🔍 What does this kernel do?
This is a sharpening filter. It emphasizes edges and fine details by subtracting surrounding pixels and boosting the center pixel.

📌 How it works:
The center value 5 strengthens the current pixel.

The surrounding -1 values subtract neighboring pixel values (detecting change).

This enhances differences (edges), making text and borders crisper.

In [43]:
edges = cv2.Canny(gray, 100, 200)


In [44]:
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


In [45]:
def extract_ethiopian_plate(ocr_text):
    """
    Extract structured Ethiopian license plate data.
    Format: [digit][Ethiopic Letter][3-6 digits], e.g., 1ነ23456
    """
    cleaned = re.sub(r'[^0-9\u1200-\u137F]', '', ocr_text)

    match = re.match(r'(\d)([\u1200-\u137F])(\d{3,6})', cleaned)
    if match:
        region, letter, number = match.groups()
        return {
            "region_code": region,
            "letter": letter,
            "serial_number": number,
            "formatted": f"{region} {letter} {number}"
        }
    return {
        "region_code": None,
        "letter": None,
        "serial_number": None,
        "formatted": cleaned
    }

In [46]:
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w / h

    if 2 < aspect_ratio < 6 and w > 100:
        plate = img[y:y+h, x:x+w]
        ocr_raw = pytesseract.image_to_string(plate, config='--psm 7')

        # Process plate format
        plate_info = extract_ethiopian_plate(ocr_raw)
        print("Raw OCR:", ocr_raw)
        print("Extracted Plate:", plate_info["formatted"])

        # Draw rectangle and label
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img, plate_info["formatted"], (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        plate_found = True
        break  # Stop after first valid plate (optional)

if not plate_found:
    print("No plate detected.")

# Show the final image with bounding box and label
cv2.imshow("Detected Plate", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Raw OCR: (@nB79542A)

Extracted Plate: 79542


In [48]:
import cv2
import numpy as np
import easyocr
import re

# Initialize EasyOCR with Amharic and English support
reader = easyocr.Reader(['en'])

def extract_ethiopian_plate(ocr_text):
    """
    Extract structured Ethiopian license plate data.
    Format: [digit][Ethiopic Letter][3-6 digits], e.g., 1ነ23456
    """
    cleaned = re.sub(r'[^0-9\u1200-\u137F]', '', ocr_text)

    match = re.match(r'(\d)([\u1200-\u137F])(\d{3,6})', cleaned)
    if match:
        region, letter, number = match.groups()
        return {
            "region_code": region,
            "letter": letter,
            "serial_number": number,
            "formatted": f"{region} {letter} {number}"
        }
    return {
        "region_code": None,
        "letter": None,
        "serial_number": None,
        "formatted": cleaned
    }

# Load the image
img = cv2.imread("C:\\Users\\Aman\\Desktop\\thesisproject\\car_image\\Ethiopia_EV1.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Preprocessing: denoise, enhance contrast, sharpen
# gray = cv2.bilateralFilter(gray, 11, 17, 17)
# gray = cv2.equalizeHist(gray)
# sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
# gray = cv2.filter2D(gray, -1, sharpen_kernel)

# Edge detection
edges = cv2.Canny(gray, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

plate_found = False

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w / h

    if 2 < aspect_ratio < 6 and w > 100:
        plate_img = img[y:y+h, x:x+w]

        # Resize if too small
        if plate_img.shape[1] < 200:
            plate_img = cv2.resize(plate_img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

        # OCR with EasyOCR
        results = reader.readtext(plate_img)

        if results:
            raw_text = results[0][1]
            plate_info = extract_ethiopian_plate(raw_text)
            print("Raw OCR:", raw_text)
            print("Extracted Plate:", plate_info["formatted"])

            # Draw rectangle and label
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, plate_info["formatted"], (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            plate_found = True
            break

if not plate_found:
    print("No plate detected.")

# Show result
cv2.imshow("Detected Plate", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Raw OCR: 6B795421
Extracted Plate: 6795421
